In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import mplhep as hep

plt.style.use(hep.style.CMS)
hep.style.use("CMS")
formatter = mticker.ScalarFormatter(useMathText=True)
formatter.set_powerlimits((-3, 3))

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import mplhep as hep

# Apply the CMS style
plt.style.use(hep.style.CMS)

# Define the observed and expected data for the first set (from the first PDF)
observed_value_1 = 142
expected_median_1 = 69
expected_68_low_1, expected_68_high_1 = 50, 80  # 68% confidence interval
expected_95_low_1, expected_95_high_1 = 40, 100  # 95% confidence interval

# Define the observed and expected data for the second set (from the second PDF)
observed_value_2 = 1.1
expected_median_2 = 0.9
expected_68_low_2, expected_68_high_2 = 0.7, 1.1  # 68% confidence interval
expected_95_low_2, expected_95_high_2 = 0.5, 1.3  # 95% confidence interval

# Create the figure with custom dimensions
fig, ax = plt.subplots(figsize=(10, 6))  # Adjust the dimensions as needed

# Plot for the first set
# Add rectangles for the confidence intervals
rect_95_1 = patches.Rectangle(
    (expected_95_low_1, 0.7),
    expected_95_high_1 - expected_95_low_1,
    0.2,
    linewidth=0,
    edgecolor="none",
    facecolor="yellow",
    alpha=0.5,
)
ax.add_patch(rect_95_1)
rect_68_1 = patches.Rectangle(
    (expected_68_low_1, 0.7),
    expected_68_high_1 - expected_68_low_1,
    0.2,
    linewidth=0,
    edgecolor="none",
    facecolor="green",
    alpha=0.5,
)
ax.add_patch(rect_68_1)
ax.plot([expected_median_1, expected_median_1], [0.7, 0.9], "k--")
ax.plot([observed_value_1, observed_value_1], [0.7, 0.9], "k-", linewidth=2)

# Plot for the second set
# Add rectangles for the confidence intervals
rect_95_2 = patches.Rectangle(
    (expected_95_low_2, 0.4),
    expected_95_high_2 - expected_95_low_2,
    0.2,
    linewidth=0,
    edgecolor="none",
    facecolor="yellow",
    alpha=0.5,
)
ax.add_patch(rect_95_2)
rect_68_2 = patches.Rectangle(
    (expected_68_low_2, 0.4),
    expected_68_high_2 - expected_68_low_2,
    0.2,
    linewidth=0,
    edgecolor="none",
    facecolor="green",
    alpha=0.5,
)
ax.add_patch(rect_68_2)
ax.plot([expected_median_2, expected_median_2], [0.4, 0.6], "k--")
ax.plot([observed_value_2, observed_value_2], [0.4, 0.6], "k-", linewidth=2)

# Set the x and y axis labels
ax.set_xlabel(r"95% CL limit on $\sigma(pp \rightarrow HH) / \sigma$")
ax.set_yticks([0.8, 0.5])
ax.set_yticklabels(
    [
        r"$\kappa_{\lambda} = 1, \kappa_{t} = 2, \kappa_{V} = 1$",
        r"$\kappa_{\lambda} = 1, \kappa_{t} = 1, \kappa_{V} = 0$",
    ]
)

# Set the title
ax.set_title("CMS Work in Progress")

# Set x-axis to logarithmic scale
ax.set_xscale("log")

# Add a legend in the top right without the limit values
legend_elements = [
    patches.Patch(color="green", alpha=0.5, label="68% expected"),
    patches.Patch(color="yellow", alpha=0.5, label="95% expected"),
    plt.Line2D([0], [0], color="k", linestyle="--", label="Median expected"),
    plt.Line2D([0], [0], color="k", linewidth=2, label="Observed"),
]
ax.legend(handles=legend_elements, loc="upper right")

# Set x-axis limits
ax.set_xlim(0.1, 200)

# Use scientific notation for the x-axis
ax.xaxis.set_major_formatter(plt.ScalarFormatter(useMathText=True))
ax.ticklabel_format(style="sci", axis="x", scilimits=(0, 0))

# Show grid
ax.grid(True, axis="x")

# Apply CMS label with `mplhep`
hep.cms.label(ax=ax, data=True, lumi=138, com=13)

# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Define the observed and expected data
observed_value = 142
expected_median = 69
expected_68_low, expected_68_high = 50, 80  # 68% confidence interval
expected_95_low, expected_95_high = 40, 100  # 95% confidence interval

# Create the plot with custom dimensions
fig, ax = plt.subplots(figsize=(8, 2))  # Adjust the dimensions as needed

# Add rectangles for the confidence intervals
# 95% confidence interval
rect_95 = patches.Rectangle(
    (expected_95_low, -0.1),
    expected_95_high - expected_95_low,
    0.2,
    linewidth=0,
    edgecolor="none",
    facecolor="yellow",
    alpha=0.5,
    label="95% expected",
)
ax.add_patch(rect_95)

# 68% confidence interval
rect_68 = patches.Rectangle(
    (expected_68_low, -0.1),
    expected_68_high - expected_68_low,
    0.2,
    linewidth=0,
    edgecolor="none",
    facecolor="green",
    alpha=0.5,
    label="68% expected",
)
ax.add_patch(rect_68)

# Plot the expected median line
ax.plot([expected_median, expected_median], [-0.1, 0.1], "k--", label="Median expected: 69")

# Plot the observed value as a solid black line
ax.plot([observed_value, observed_value], [-0.1, 0.1], "k-", linewidth=2, label="Observed: 142")

# Set the x and y axis labels
ax.set_xlabel(r"95% CL limit on $\sigma(pp \rightarrow HH) / \sigma$")
ax.set_yticks([])  # Remove y-axis ticks

# Set the title
ax.set_title("CMS Work in Progress")

# Add a legend
ax.legend()

# Set x-axis limits
ax.set_xlim(0, 150)  # Adjust based on the range of your data
ax.set_ylim(-0.2, 0.3)  # Adjust y-axis to provide space for the kappa values

# Use scientific notation for the x-axis
ax.xaxis.set_major_formatter(plt.ScalarFormatter(useMathText=True))
ax.ticklabel_format(style="sci", axis="x", scilimits=(0, 0))

# Add kappa values above the observed limit within the figure box
kappa_values = r"$\kappa_{\lambda} = 1, \kappa_{t} = 2, \kappa_{V} = 1$"
plt.text(
    observed_value,
    0.2,
    kappa_values,
    fontsize=12,
    horizontalalignment="center",
    verticalalignment="bottom",
)

# Show grid
ax.grid(True, axis="x")

# Show the plot
plt.show()